# Load Shanghai-Tech Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive/part_B_final"

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from scipy.io import loadmat
from matplotlib.colors import LogNorm
import matplotlib.cm as cm
import time
import skimage.measure
import random
import os
from PIL import Image,ImageFilter,ImageDraw,ImageStat
import numpy as np
import h5py
import cv2
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as F
import shutil
import torch.nn as nn
from torchvision import models
import scipy.io as io
import PIL.Image as Image
import glob
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
import scipy
import json
from matplotlib import cm as CM

%matplotlib inline


## Example of image in the Dataset

In [ ]:
path = ".../test_data/images/"
for name in sorted(os.listdir(path = path))[3:]:
    print(name)
    img = cv2.cvtColor(cv2.imread(path + name), cv2.COLOR_BGR2RGB)
    break

_ = plt.imshow(img)

In [ ]:
path = ".../test_data/ground_truth/"
for name in sorted(os.listdir(path = path))[3:]:
    print(name)
    heads = loadmat(path + name)
    break


plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(img)
print(f"Number of people: {len(heads['image_info'][0, 0][0, 0][0])}")

for x,y in heads["image_info"][0, 0][0, 0][0]:
    plt.scatter(round(x),round(y), s = 50, marker = "x", color = "r")

## Definition of the Density Map Function

In [ ]:
def density_map(image, heads):
    output = np.zeros(image.shape[:-1])
    x = heads["image_info"][0, 0][0, 0][0][:,0]
    y = heads["image_info"][0, 0][0, 0][0][:,1]

    for x_choord, y_choord in zip(x, y):
        density_i = np.zeros(image.shape[:-1])
        try:
            density_i[round(y_choord),round(x_choord)] = 1
        except:
            pass
        nearest_distance = np.min(np.linalg.norm(x_choord - y_choord))
        sigma = 0.3 * nearest_distance
        kernel_size = 3

        density_i = cv2.GaussianBlur(density_i, (kernel_size,kernel_size), sigma)
        output += density_i
    return cv2.GaussianBlur(output, (51,51), sigma)

start = time.time()
ds_map = density_map(img, heads)
print(f"Time: {time.time() - start}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.axis("off")
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.axis("off")
_=plt.imshow(ds_map , cmap = cm.jet)

## Downsampled version 1/8

In [ ]:
def density_map_1_over_8(image, heads):
    image = skimage.measure.block_reduce(image, (2,2,1), np.max)
    image = skimage.measure.block_reduce(image, (2,2,1), np.max)
    image = skimage.measure.block_reduce(image, (2,2,1), np.max)

    output = np.zeros(image.shape[:-1])
    x = heads["image_info"][0, 0][0, 0][0][:,0]/8
    y = heads["image_info"][0, 0][0, 0][0][:,1]/8

    for x_choord, y_choord in zip(x, y):
        density_i = np.zeros(image.shape[:-1])
        try:
            density_i[round(y_choord),round(x_choord)] = 1
        except:
            pass
        nearest_distance = np.min(np.linalg.norm(x_choord - y_choord))
        sigma = 0.3 * nearest_distance # beta = 0.3 suggested by MCNN paper
        kernel_size = 3

        density_i = cv2.GaussianBlur(density_i, (kernel_size,kernel_size), sigma)
        output += density_i
    return cv2.GaussianBlur(output, (5,5), sigma)

start = time.time()
ds_map = density_map_1_over_8(img, heads)
print(f"Time: {time.time() - start}")

In [ ]:

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.axis("off")
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.axis("off")
_=plt.imshow(ds_map, cmap = cm.jet)

# Conversion

Train:

In [ ]:
import os
import cv2
from scipy.io import loadmat
import numpy as np

path_images = ".../train_data/images/"
path_heads = ".../train_data/ground_truth/"
path_save = ".../train_data/maps/"

for i, (img_name, heads_name) in enumerate(zip(sorted(os.listdir(path=path_images)), sorted(os.listdir(path=path_heads)))):
    img = cv2.cvtColor(cv2.imread(path_images + img_name), cv2.COLOR_BGR2RGB)
    heads = loadmat(path_heads + heads_name)

    ds_map_1o8 = density_map_1_over_8(img, heads)

    np.save(path_save + img_name[:-4], ds_map_1o8)

    if i % 10 == 0:
        print(f"{i+1} density maps done")

print(f"Total {i+1} density maps generated.")


Test:


In [ ]:
import os
import cv2
from scipy.io import loadmat
import numpy as np

path_images = ".../test_data/images/"
path_heads = ".../test_data/ground_truth/"
path_save = ".../test_data/maps/"

for i, (img_name, heads_name) in enumerate(zip(sorted(os.listdir(path=path_images)), sorted(os.listdir(path=path_heads)))):
    img = cv2.cvtColor(cv2.imread(path_images + img_name), cv2.COLOR_BGR2RGB)
    heads = loadmat(path_heads + heads_name)

    ds_map_1o8 = density_map_1_over_8(img, heads)

    np.save(path_save + img_name[:-4], ds_map_1o8)

    if i % 10 == 0:
        print(f"{i+1} density maps done")

print(f"Total {i+1} density maps generated for the test data.")


## Print some

In [ ]:
to_print = 5
plt.figure(figsize=(6, 3 * to_print))

for i, (img_name, den_name) in enumerate(zip(sorted(os.listdir(path = path_images)), sorted(os.listdir(path = path_save)))):
    if i == to_print: break

    img = cv2.cvtColor(cv2.imread(path_images + img_name), cv2.COLOR_BGR2RGB)
    ds_map_1o8 = np.load(path_save + den_name)
    print(img_name,den_name)
    plt.subplot(to_print, 2, i * 2 + 1)
    plt.axis("off")
    plt.imshow(img)
    plt.subplot(to_print, 2, (i+1) * 2)
    plt.axis("off")
    plt.imshow(ds_map_1o8 , cmap = cm.jet)